In [1]:
import pandas as pd
import numpy as np
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import string
from scipy.sparse import coo_matrix

## Считаем информацию о ссылках из файла

In [2]:
f = open('Cit-HepTh.txt', 'r')
l = [line.strip() for line in f]

In [3]:
l=l[4:]
for i in range(0,len(l)):
    l[i]=l[i].replace('\t',' ')
    l[i]=l[i].split(' ')

### Составим разреженную матрицу смежности графа

In [4]:
#l=[[2,4],[1,2],[2,1]]
row=[]
col=[]
for i in range(0,len(l)):
    row.append(l[i][0])
    col.append(l[i][1])


In [5]:
row = np.array(row)
col = np.array(col)
data = np.ones(len(l))
Mat = coo_matrix((data, (row, col)), shape=(9912294, 9912294)) #9912294

### Найдем количество ребер, выходящих из каждой вершины

In [6]:
a=Mat.sum(1)
print(a[0:5,0])



[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


### Построим матрицу перехода P

In [7]:
row=[]
col=[]
data=[]
for i in range(0,len(l)):
    col.append(l[i][0])
    row.append(l[i][1])
    k=int(l[i][0])
    if(a[k,0]==0):
        #print(l[i][0],l[i][1],a[k,0])
        data.append(1/5)
    else:
        #print(l[i][0],l[i][1],1/a[k,0])
        data.append(1/a[k,0])
row = np.array(row)
col = np.array(col)
data = np.array(data)
P = coo_matrix((data, (row, col)), shape=(9912294, 9912294))

In [8]:
P.sum(0)

matrix([[0., 0., 0., ..., 1., 1., 1.]])

In [13]:
r = np.array([1/9912294 for i in range(0,9912294)])
r = np.transpose(r)
r

array([1.0088482e-07, 1.0088482e-07, 1.0088482e-07, ..., 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07])

In [14]:
previous_r = r
for i in range(1,10000):
   r = P * r
   #print (display_format(r,3))
   if (np.linalg.norm(previous_r-r)<10**(-5)).all():
      break
   previous_r = r
#print ("Final:\n", display_format(r,3))
print ("sum", np.sum(r))

sum 0.0007318959902597282


In [15]:
sorted(enumerate(r),key=lambda x: x[1], reverse=True)[:10]

[(9207016, 1.2221681535272335e-05),
 (9201015, 1.1089090980618818e-05),
 (9407087, 1.1062583248071842e-05),
 (9503124, 9.906111017426891e-06),
 (9402044, 9.849104756545422e-06),
 (9510017, 8.106737343844373e-06),
 (9410167, 8.038621253910471e-06),
 (9205027, 7.636351846337389e-06),
 (9402002, 7.534982560766894e-06),
 (9207053, 6.885224009577918e-06)]

### По результатам хочется сказать,что возникли проблемы с реализацией разреженной матрицы. А именно в момент составления матрицы Р, когда для вершины, из которой не исходит ребер мы должны в соответствующий ей столбец поставить дефолтовые значения $\frac{1}{N}$, но таких вершин дофига, тогда матрица не будет разреженной и питон будет ругаться, что я пытаюсь забить много памяти, поэтому в этой реализации они проинициализированны 0, что конечно же, могло повлиять на конечный результат.